In [0]:
df_silver = spark.table("workspace.silver.flights_delay_limpos")
display(df_silver.limit(5))


In [0]:
from pyspark.sql.functions import col, avg, count, round

df_gold = (
    df_silver
    .groupBy("origin")
    .agg(
        round(avg(col("delay")), 2).alias("media_atraso_minutos"),
        count("*").alias("total_voos")
    )
    .orderBy(col("media_atraso_minutos").desc())
)

display(df_gold.limit(10))


In [0]:
df_gold.write.format("delta") \
    .option("mergeSchema", "true") \
    .mode("overwrite") \
    .saveAsTable("workspace.gold.flights_metricas")


In [0]:
%sql
SELECT * FROM workspace.gold.flights_metricas ORDER BY media_atraso_minutos DESC LIMIT 10;


In [0]:
%sql
OPTIMIZE workspace.gold.flights_metricas ZORDER BY (origin);